# Playlist analyzer

Because of what happened with my account being flagged, this current code doesn't work. It was working perfectly until class, but I will have to find a workaround that doesn't use the api to find song data on the user's playlist. 

## Import the spotify api

In [19]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

# Define your credentials and redirection URI
SPOTIPY_CLIENT_ID = 'dda606479dd244a9b0948c01f155ac7a'
SPOTIPY_CLIENT_SECRET = 'd714739739a04b0fa662e521e1a85ff8'
SPOTIPY_REDIRECT_URI = 'http://localhost:3000'

# Create the Spotify client

spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIPY_CLIENT_ID,
    client_secret=SPOTIPY_CLIENT_SECRET,
    redirect_uri=SPOTIPY_REDIRECT_URI,
))

user_name = input("Enter your spotify ID: ")
playlists = spotify.user_playlists(user_name)
name = input("Enter playlist name: ")


# https://stackoverflow.com/questions/39086287/spotipy-how-to-read-more-than-100-tracks-from-a-playlist
def get_playlist_tracks(playlist_id):
    results = spotify.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = spotify.next(results)
        tracks.extend(results['items'])
    return tracks

playlist_id = 0
for i in playlists["items"] : 
    if i["name"] == name :
        playlist_id = i["id"]
playlist_data = get_playlist_tracks(playlist_id)


## Extract song data on each song

In [20]:
playlist = []
names = []

for i in playlist_data : 
    names.append(i["track"]["name"])
    playlist.append(spotify.audio_features(i["track"]["id"])[0])

print(playlist[1])


Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=2zyLMphhwqYIYhGvh36SoG:
 Max Retries, reason: too many 429 error responses

### Convert the new data into a dataframe including the name of each song

In [21]:
collumns = ["danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "type", "id", "uri", "track_href", "analysis_url", "duration_ms", "time_signature"]
selected_columns = ["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]

df = pd.DataFrame(playlist, columns=collumns)
df = df[selected_columns]
df["name"] = names

scaler = MinMaxScaler()
scaler = scaler.fit(df[["tempo"]])

df["tempo"] = pd.DataFrame(scaler.transform(df[["tempo"]]))
df





/Users/liamkrenz/Desktop/ATCS/Final Project/ATCSFinalProject/.venv/lib/python3.9/site-packages/sklearn/utils/_array_api.py:472: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/Users/liamkrenz/Desktop/ATCS/Final Project/ATCSFinalProject/.venv/lib/python3.9/site-packages/sklearn/utils/_array_api.py:489: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,name
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Knew You


## Find the distance of each song to the central point of the playlist's cluster

In [ ]:
model = KMeans(n_clusters=1)
model.fit(df[selected_columns])


centroid = model.cluster_centers_[0]
distances = []

for i in range(len(df)) :
    distance = cdist([df[selected_columns].iloc[i]], [centroid], metric="euclidean")
    distances.append(distance[0][0])

df["distance"] = distances

Q1 = df["distance"].quantile(0.25)
Q3 = df['distance'].quantile(0.75)

max = Q3 + (Q3-Q1)*1.5
print(max)

df.loc[df["distance"] > max]



4.731741198633519


,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,name,distance
51,0.566,0.366,-12.808,0.0280,0.113,0.18100,0.155,0.237,0.506634,we fell in love in october,6.704960
80,0.809,0.463,-11.377,0.0396,0.256,0.08500,0.109,0.364,0.360727,Passionfruit,5.270838
123,0.453,0.350,-11.639,0.0425,0.827,0.00122,0.144,0.279,0.036428,Arcade,5.586864
158,0.226,0.280,-11.460,0.0316,0.949,0.08870,0.315,0.625,0.221011,The Star Spangled Banner (United States of Ame...,5.431824


## Load in the data

In [ ]:
df = pd.read_csv("data/bike_share.csv")

## Find recommendations